In [1]:
""" Replicating the SimpleNet model architecture. """

import keras
import numpy as np
import keras.backend as K
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.models import Sequential
from keras.models import load_model
from keras import regularizers, optimizers
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import glorot_normal, RandomNormal, Zeros
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization

Using TensorFlow backend.


In [2]:
# Data Retrieval & mean/std preprocess
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [3]:
# Data augmentation
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
    )
datagen.fit(x_train)

In [4]:
# Define Model architecture
def create_model(s = 2, weight_decay = 1e-2, act="relu"):
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal(), input_shape=x_train.shape[1:]))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 2
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 3
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 4
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    # First Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))
    
    
    # Block 5
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 6
    model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 7
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    # Second Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    
    # Block 8
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 9
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    # Third Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    
    
    # Block 10
    model.add(Conv2D(512, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))

    # Block 11  
    model.add(Conv2D(2048, (1,1), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    # Block 12  
    model.add(Conv2D(256, (1,1), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    # Fourth Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))


    # Block 13
    model.add(Conv2D(256, (3,3), padding='same', kernel_regularizer=regularizers.l2(0.005), kernel_initializer=glorot_normal()))
    model.add(Activation(act))
    # Fifth Maxpooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))

    # Final Classifier
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [5]:
# Prepare for training 
model = create_model(act="relu")
batch_size = 128
epochs = 25
train = {}

# First training for 50 epochs - (0-50)
opt_adm = keras.optimizers.Adadelta(lr=0.1)
model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
train["part_1"] = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=2*epochs,
                                    verbose=1,validation_data=(x_test,y_test))
model.save("simplenet_generic_first.h5")
print(train["part_1"].history)

Epoch 1/50
390/390 [==============================] - 87s 223ms/step - loss: 13.3755 - acc: 0.3261 - val_loss: 12.1981 - val_acc: 0.165813 - ac - ETA: 18s - loss: 13.9671 - acc  - ETA: 8s - loss: 13.66 - ETA: 7s - l - ETA: 0s - loss: 13.3826 - acc: 0.32
Epoch 2/50
390/390 [==============================] - 81s 207ms/step - loss: 8.7435 - acc: 0.4899 - val_loss: 8.1817 - val_acc: 0.2400
Epoch 3/50
390/390 [==============================] - 80s 205ms/step - loss: 5.7768 - acc: 0.5936 - val_loss: 5.5061 - val_acc: 0.40195.8794 - acc: 0. - ETA: 6s - loss: 5.8682 - acc: 0. - ETA: 6s  - ETA: 0s - loss: 5.7852 - acc: 0.5
Epoch 4/50
390/390 [==============================] - 80s 206ms/step - loss: 4.0173 - acc: 0.6623 - val_loss: 3.6880 - val_acc: 0.5883
Epoch 5/50
390/390 [==============================] - 80s 206ms/step - loss: 2.9550 - acc: 0.7098 - val_loss: 2.6577 - val_acc: 0.7044
Epoch 6/50
390/390 [==============================] - 80s 206ms/step - loss: 2.3053 - acc: 0.7417 - val_loss

In [6]:
# Training for 25 epochs more - (50-75)
opt_adm = keras.optimizers.Adadelta(lr=0.1*0.1)
model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
train["part_2"] = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,
                                    verbose=1,validation_data=(x_test,y_test))
model.save("simplenet_generic_second.h5")
print(train["part_2"].history)

Epoch 1/25
390/390 [==============================] - 83s 213ms/step - loss: 0.5279 - acc: 0.9294 - val_loss: 0.5352 - val_acc: 0.9265
Epoch 2/25
390/390 [==============================] - 80s 205ms/step - loss: 0.4940 - acc: 0.9394 - val_loss: 0.5299 - val_acc: 0.9269
Epoch 3/25
390/390 [==============================] - 80s 205ms/step - loss: 0.4794 - acc: 0.9415 - val_loss: 0.5272 - val_acc: 0.9278
Epoch 4/25
390/390 [==============================] - 80s 205ms/step - loss: 0.4667 - acc: 0.9459 - val_loss: 0.5060 - val_acc: 0.9317
Epoch 5/25
390/390 [==============================] - 80s 205ms/step - loss: 0.4577 - acc: 0.9473 - val_loss: 0.5014 - val_acc: 0.9316.4567 - acc:  - ETA: 4s - loss: 0.4566 - ac - ETA: 2s - loss: 0.4570 
Epoch 6/25
390/390 [==============================] - 80s 205ms/step - loss: 0.4502 - acc: 0.9488 - val_loss: 0.4975 - val_acc: 0.9321
Epoch 7/25
390/390 [==============================] - 80s 205ms/step - loss: 0.4397 - acc: 0.9497 - val_loss: 0.4869 - va

In [7]:
# Training for 25 epochs more - (75-100)
opt_adm = keras.optimizers.Adadelta(lr=0.1*0.1*0.1)
model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
train["part_3"] = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,
                                    verbose=1,validation_data=(x_test,y_test))
model.save("simplenet_generic_third.h5")
print(train["part_3"].history)

Epoch 1/25
199/390 [==============>...............] - ETA: 40s - loss: 0.3423 - acc: 0.9675 - ETA: 53s - loss: 0.3407 - acc: 0.96 - - ETA: 48s - loss: 0.3399 - acc - ETA: 47s - lo

MemoryError: 

In [ ]:
# Training for 25 epochs more  - (100-125)
opt_adm = keras.optimizers.Adadelta(lr=0.1*0.1*0.1*0.1)
model.compile(loss='categorical_crossentropy', optimizer=opt_adm, metrics=['accuracy'])
train["part_4"] = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,
                                    verbose=1,validation_data=(x_test,y_test))
model.save("simplenet_generic_fourth.h5")
print(train["part_4"].history)

In [ ]:
print("\n \n Final Logs: ", train)

In [ ]:
# Done!